# K Nearest Neighbor Worksheet

We are going to look at the following ratings:

|Customer | Taylor Swift | Miranda Lambert | Carrie Underwood | Jhené Aiko | Tinashe | Kelela | Randy Wood |
|:-----------|:------:|:------:|:---------:|:------:|:--------:|:------:|:--------:|
| Sarah | 5 | | 5 | 2 | | 2 |  |
| Miquel | 2 | |   | 4 |  5 | 5  | 1 |
| Tyler | 4 | 4 | 5 | 2 | 1 | 2 | 1  |
| Ann | 2 | 3 |   | 5 | | 5 | 1 |
| Jessica | 2 | 1 |   | 5 | |   | 1 |
| Franklin | 5 | 5 | 5 |   | 2 | 3 | 2  |
| Jenny | 4 | 4 | 4 | 1 | 1 | 1 | 5 |

Spend a minute and look this table over. What do you think will be our highest recommendation for Sarah?

We will implement this data in Python as follows:

In [16]:
ratings = {'Sarah':  {'Taylor Swift': 5, 'Carrie Underwood': 5, 'Jhené Aiko': 2, 'Kelela': 2},
           'Miguel': {'Taylor Swift': 2, 'Jhené Aiko': 4, 'Kelela': 5, 'Tinashe': 5, 'Randy Wood': 1},
           'Tyler': {'Taylor Swift': 4, 'Miranda Lambert': 4, 'Carrie Underwood': 5, 'Jhené Aiko': 2, 'Kelela': 2, 'Tinashe': 1, 'Randy Wood': 1},
           'Ann': {'Taylor Swift': 2, 'Miranda Lambert': 3,  'Jhené Aiko': 5, 'Kelela': 5,  'Randy Wood': 1},
           'Jessica': {'Taylor Swift': 2, 'Miranda Lambert': 1,  'Jhené Aiko': 5, 'Randy Wood': 1},
           'Franklin': {'Taylor Swift': 4, 'Miranda Lambert': 5, 'Carrie Underwood': 5, 'Jhené Aiko': 2, 'Kelela': 3, 'Tinashe': 2, 'Randy Wood': 2},
           'Jenny': {'Taylor Swift': 4, 'Miranda Lambert': 4, 'Carrie Underwood': 4, 'Jhené Aiko': 1, 'Kelela': 1, 'Tinashe': 1, 'Randy Wood': 5}
          }

Here is the code we used in lab 1:

In [18]:
def manhattan(rating1, rating2):
    """Computes the Manhattan distance. Both rating1 and rating2 are dictionaries
       of the form {'The Strokes': 3.0, 'Slightly Stoopid': 2.5}"""
    distance = 0
    commonRatings = False 
    for key in rating1:
        if key in rating2:
            distance += abs(rating1[key] - rating2[key])
            commonRatings = True
    if commonRatings:
        return distance
    else:
        return -1 #Indicates no ratings in common
    
def computeNearestNeighbor(username, users):
    """creates a sorted list of users based on their distance to username"""
    distances = []
    for user in users:
        if user != username:
            distance = manhattan(users[user], users[username])
            distances.append((distance, user))
    # sort based on distance -- closest first
    distances.sort()
    return distances

def recommend(username, users):
    """Give list of recommendations"""
    # first find nearest neighbor
    nearest = computeNearestNeighbor(username, users)[0][1]

    recommendations = []
    # now find bands neighbor rated that user didn't
    neighborRatings = users[nearest]
    userRatings = users[username]
    for artist in neighborRatings:
        if not artist in userRatings:
            recommendations.append((artist, neighborRatings[artist]))
    # using the fn sorted for variety - sort is more efficient
    return sorted(recommendations, key=lambda artistTuple: artistTuple[1], reverse = True)

### checkpoint 1
What does this system recommend for Sarah?

So the highest rating for Sarah is the artist you discovered above. Let's examine a different method.

## What is the difference?



Here is a long code block defining the recommender class:

In [2]:
import codecs
from math import sqrt

class recommender:

    def __init__(self, data, k=1, metric='pearson', n=5):
        """ initialize recommender
        currently, if data is dictionary the recommender is initialized
        to it.
        For all other data types of data, no initialization occurs
        k is the k value for k nearest neighbor
        metric is which distance formula to use
        n is the maximum number of recommendations to make"""
        self.k = k
        self.n = n
        self.username2id = {}
        self.userid2name = {}
        self.productid2name = {}
        # for some reason I want to save the name of the metric
        self.metric = metric
        if self.metric == 'pearson':
            self.fn = self.pearson
        #
        # if data is dictionary set recommender data to it
        #
        if type(data).__name__ == 'dict':
            self.data = data

    def convertProductID2name(self, id):
        """Given product id number return product name"""
        if id in self.productid2name:
            return self.productid2name[id]
        else:
            return id


    def userRatings(self, id, n):
        """Return n top ratings for user with id"""
        print ("Ratings for " + self.userid2name[id])
        ratings = self.data[id]
        print(len(ratings))
        ratings = list(ratings.items())
        ratings = [(self.convertProductID2name(k), v)
                   for (k, v) in ratings]
        # finally sort and return
        ratings.sort(key=lambda artistTuple: artistTuple[1],
                     reverse = True)
        ratings = ratings[:n]
        for rating in ratings:
            print("%s\t%i" % (rating[0], rating[1]))
        

        

    def loadBookDB(self, path=''):
        """loads the BX book dataset. Path is where the BX files are
        located"""
        self.data = {}
        i = 0
        #
        # First load book ratings into self.data
        #
        f = codecs.open(path + "BX-Book-Ratings.csv", 'r', 'utf8')
        for line in f:
            i += 1
            #separate line into fields
            fields = line.split(';')
            user = fields[0].strip('"')
            book = fields[1].strip('"')
            rating = int(fields[2].strip().strip('"'))
            if user in self.data:
                currentRatings = self.data[user]
            else:
                currentRatings = {}
            currentRatings[book] = rating
            self.data[user] = currentRatings
        f.close()
        #
        # Now load books into self.productid2name
        # Books contains isbn, title, and author among other fields
        #
        f = codecs.open(path + "BX-Books.csv", 'r', 'utf8')
        for line in f:
            i += 1
            #separate line into fields
            fields = line.split(';')
            isbn = fields[0].strip('"')
            title = fields[1].strip('"')
            author = fields[2].strip().strip('"')
            title = title + ' by ' + author
            self.productid2name[isbn] = title
        f.close()
        #
        #  Now load user info into both self.userid2name and
        #  self.username2id
        #
        f = codecs.open(path + "BX-Users.csv", 'r', 'utf8')
        for line in f:
            i += 1
            #print(line)
            #separate line into fields
            fields = line.split(';')
            userid = fields[0].strip('"')
            location = fields[1].strip('"')
            if len(fields) > 3:
                age = fields[2].strip().strip('"')
            else:
                age = 'NULL'
            if age != 'NULL':
                value = location + '  (age: ' + age + ')'
            else:
                value = location
            self.userid2name[userid] = value
            self.username2id[location] = userid
        f.close()
        print(i)
                
        
    def pearson(self, rating1, rating2):
        sum_xy = 0
        sum_x = 0
        sum_y = 0
        sum_x2 = 0
        sum_y2 = 0
        n = 0
        for key in rating1:
            if key in rating2:
                n += 1
                x = rating1[key]
                y = rating2[key]
                sum_xy += x * y
                sum_x += x
                sum_y += y
                sum_x2 += pow(x, 2)
                sum_y2 += pow(y, 2)
        if n == 0:
            return 0
        # now compute denominator
        denominator = (sqrt(sum_x2 - pow(sum_x, 2) / n)
                       * sqrt(sum_y2 - pow(sum_y, 2) / n))
        if denominator == 0:
            return 0
        else:
            return (sum_xy - (sum_x * sum_y) / n) / denominator


    def computeNearestNeighbor(self, username):
        """creates a sorted list of users based on their distance to
        username"""
        distances = []
        for instance in self.data:
            if instance != username:
                distance = self.fn(self.data[username],
                                   self.data[instance])
                distances.append((instance, distance))
        # sort based on distance -- closest first
        distances.sort(key=lambda artistTuple: artistTuple[1],
                       reverse=True)
        return distances

    def recommend(self, user):
       """Give list of recommendations"""
       recommendations = {}
       # first get list of users  ordered by nearness
       nearest = self.computeNearestNeighbor(user)
       #
       # now get the ratings for the user
       #
       userRatings = self.data[user]
       #
       # determine the total distance
       totalDistance = 0.0
       for i in range(self.k):
          totalDistance += nearest[i][1]
       # now iterate through the k nearest neighbors
       # accumulating their ratings
       for i in range(self.k):
          # compute slice of pie 
          weight = nearest[i][1] / totalDistance
          # get the name of the person
          name = nearest[i][0]
          # get the ratings for this person
          neighborRatings = self.data[name]
          # get the name of the person
          # now find bands neighbor rated that user didn't
          for artist in neighborRatings:
             if not artist in userRatings:
                if artist not in recommendations:
                   recommendations[artist] = (neighborRatings[artist]
                                              * weight)
                else:
                   recommendations[artist] = (recommendations[artist]
                                              + neighborRatings[artist]
                                              * weight)
       # now make list from dictionary
       recommendations = list(recommendations.items())
       recommendations = [(self.convertProductID2name(k), v)
                          for (k, v) in recommendations]
       # finally sort and return
       recommendations.sort(key=lambda artistTuple: artistTuple[1],
                            reverse = True)
       # Return the first n items
       return recommendations[:self.n]

We will look at that in more detail in a bit. First, let's make a simple recommendation. First I am going to make an instance of this recommender:

In [11]:
r = recommender(ratings)

Now let's see what we should recommend to Ann:

In [20]:
r.recommend('Ann')

[('Tinashe', 2.7284445256913887), ('Carrie Underwood', -0.1773234608330154)]

Okay, it looks like Ann will really hate Carrie Underwood. 

#### checkpoint 2

Who is Sarah's nearest neighbor?


#### checkpoint 3

What artist does this system recommend for Sarah?


#### puzzler 1
Is this the same or different than the recommendation made at Checkpoint 1 above?

If it is different, why is it different? (Look at the code for the recommender class). Need a hint? See the hint section below.

#### puzzler 2
Do you see a problem with only using the nearest neighbor? What is that problem?

### the k-nearest neighbor approach
Instead of looking at a single nearest neighbor, we are going to look at the nearest *k* neighbors. For example when *k* equals 3, we will look at the three nearest neighbors.

The above code already allows for this. Can you create an instance of a recommender that uses the three nearest neighbors?

#### checkpoint 4

Great. Now who are Sarah's three nearest neighbors



#### checkpoint 5

What artists do we recommend for Sarah?

### xp 

##### 40xp for showing me all five checkpoints above.
##### extra XP for demoing (should be done on your own or with a partner - no larger groups)

1. implement Manhattan and Euclidean distance, compare results. 30xp
2. Using the book crossing dataset, add your own ratings and see what the system recommends to you. 30-40xp



